# Data Preprocessing - Weather Germany
Weather data of Germany as a whole and of all its states for mean temperature, percipitation and sunshine duration. Values are averaged for the region. In the final database, only the values for Germany will be saved, however the values for the states will be exported here as well.

In [19]:
# Installing the packages
import os
import numpy as np
import pandas as pd
import datetime

In [20]:
dataframe = []
feature = ['tm', 'rr', 'sd']
directory = ['air_temperature_mean', 'precipitation', 'sunshine_duration'] 
loc = ['be', 'bb', 'bw', 'by', 'he', 'mv', 'ni', 'hh', 'nw', 'rp', 'sh', 'sl', 
       'sn', 'st', 'st_th', 'th', 'de']
nums = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
for d, feat in enumerate(feature):
    df = None
    for i, val in enumerate(nums):
        if df is None:
            df = pd.read_table(
                f'../data/energy_households/raw_data/Wetter/'
                f'{directory[d]}/regional_averages_{feat}_{val}.txt', delimiter=';')
            # Delete the last column without wanted information
            df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
            # Set header
            df_header = df.iloc[0]
            df_data = df.iloc[1:].index
            df = pd.DataFrame([list(item) for item in df_data], 
                                columns=df_header.name)
        else:
            new_df = pd.read_table(
                f'../data/energy_households/raw_data/Wetter/'
                f'{directory[d]}/regional_averages_{feat}_{val}.txt', delimiter=';')
            # Delete the last column without wanted information
            new_df.drop(new_df.columns[len(new_df.columns)-1], axis=1,
                        inplace=True)
            # Set header
            new_df_header = new_df.iloc[0]
            new_df_data = new_df.iloc[1:].index
            new_proper_df = pd.DataFrame([list(item) for item in new_df_data], 
                            columns=new_df_header.name)
            df = df.append(new_proper_df)
    # Change date format
    df.Jahr = df.Jahr.astype('int16')
    df.Monat = df.Monat.astype('int16')
    df['date'] = df.Jahr.astype('str') + '-' + df.Monat.astype('str')
    df = df.drop(['Jahr', 'Monat'], axis=1)
    df.date = pd.to_datetime(df.date).dt.to_period('m')
    df = df.set_index('date')
    df = df.sort_index()
    # Crop old years
    df = df[df.index >= "1990-01"]

    # More readable feature names
    if feat == 'tm':
        feature_name = 'temperature_mean'
    elif feat == 'rr':
        feature_name = 'precipitation'
    else:
        feature_name = 'sunshine_duration'

    df.to_csv(f'../data/energy_households/E_Process_Weather_{feature_name}.csv')